In [111]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

import csv

mu_depth, sigma_depth = 3, 1.5
mu_delta, sigma_delta = 0.0001, 0.1

fixingfactor = 0.2
delta = []
depth = []
output = []
flag = []
noise = [4,34,37,98,25,87,123,345,233,888,567,454,222,455,653,324,54]
for k in range(1000):
    flag.append(0)
# temp2 = []
depth.append(np.random.normal(mu_depth, sigma_depth, 1000))
# depth.append(temp2)

for i in range(500):    
    delta.append(np.random.normal(mu_delta, sigma_delta, 1000))
    temp = []
    temp2 = []
    count = 0
     # fixing factor
        
    for j in range(1000):
        if flag[j] > 0:
            flag[j] -= 1
        if flag[j] <  0:
            depth[i][j] = 4
            flag[j] = 0
        if depth[i][j] - i*delta[i][j] >= 0:
            temp.append(depth[i][j] - i*delta[i][j])
        else:
            temp.append(0)
        if depth[i][j] - i*delta[i][j] < 0.5 or j in noise:
            temp2.append(1)
            flag[j] = np.random.randint(5,10) 
            count += 1
        else:
            temp2.append(0)
    
        
    depth.append(temp)
    output.append(temp2)
    
depth2 = []
depth3 = []
for i in range(1,501):
    depth2.append(np.array(depth[i]))
    depth3.append(depth[i])
    # print(count)

# print("depth:")
# print(delta)

# print("delta:")
# print(delta)

# print(output)

2.0.0-rc0


In [112]:
class_names = ['overflow', 'not overflow']

# print(np.asarray(depth2).shape)
# print(np.asarray(delta).shape)
# print(np.asarray(output).shape)

print("\n")
print(len(depth2))
print(len(delta))



500
500


In [113]:
mina = np.amin(depth2)
maxa = np.amax(depth2)
# print(np.array(depth2))
depth_norm = []

for x in depth2:
    x = (x-mina)/(maxa-mina)
    depth_norm.append(x)

mina = np.amin(delta)
maxa = np.amax(delta)

delta_norm = []

for x in delta:
    x = (x-mina)/(maxa-mina)
    delta_norm.append(x)
    
delt_train_size = int(len(delta_norm) * .8)
print("train: ",delt_train_size)
print("test: ",len(delta_norm)-delt_train_size)

dept_train_size = int(len(depth_norm) * .8)
print("train: ",dept_train_size)
print("test: ",len(depth_norm)-dept_train_size)



train:  400
test:  100
train:  400
test:  100


In [114]:
dept_train = depth_norm[:dept_train_size]
depttrain = []
delt_train = delta_norm[:delt_train_size]
delttrain = []
dept_test = depth_norm[dept_train_size:]
depttest = []
delt_test = delta_norm[delt_train_size:]
delttest = []
labels_train =  output[:dept_train_size]
labelstrain = []
labels_test = output[dept_train_size:]
labelstest = []

for i in range(1000):
    for x in dept_train:
        depttrain.append([x[i]])
    
    for x in delt_train:
        delttrain.append([x[i]])


    for x in dept_test:
        depttest.append([x[i]])
    

    for x in delt_test:
        delttest.append([x[i]])


    for x in labels_train:
        labelstrain.append(x[i])
    

    for x in labels_test:
        labelstest.append([x[i]])

In [115]:
layers = keras.layers
output2 = []

depth_inputs = layers.Input(shape=(1,))
delta_inputs = layers.Input(shape=(1,))
merged_layer = layers.concatenate([depth_inputs, delta_inputs], axis=-1)
merged_layer = layers.Dense(256, activation='relu')(merged_layer)

predictions = layers.Dense(1)(merged_layer)
deep_model = keras.Model(inputs=[depth_inputs, delta_inputs], outputs=predictions)


In [116]:

deep_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

print(deep_model.summary())

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
concatenate_11 (Concatenate)    (None, 2)            0           input_23[0][0]                   
                                                                 input_24[0][0]                   
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 256)          768         concatenate_11[0][0]      

In [ ]:
# print(np.array(delta))
# print(type(delta))
# print(type(depth2))
# print(type(output))

# print(np.asarray(delttrain).shape)
accuracies = []

for i in range(1000):
    deep_model.fit([np.asarray(delttrain[i*1000:(i+1)*1000]), np.asarray(depttrain[i*1000:(i+1)*1000])], np.asarray(labelstrain[i*1000:(i+1)*1000]), epochs=2)
    test_loss, test_acc = deep_model.evaluate([np.asarray(delttest[i*1000:(i+1)*1000]), np.asarray(depttest[i*1000:(i+1)*1000])], np.asarray(labelstest[i*1000:(i+1)*1000]))
    accuracies.append(test_acc)
# print(predictions.summary())
# deep_model.fit([dept_train], np.asarray(labels_train), epochs=10)

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 235us/sample - loss: 0.0526 - accuracy: 0.9410
Epoch 2/2
1000/1000 [==============================] - 0s 43us/sample - loss: 0.0514 - accuracy: 0.9410
1000/1 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 44us/sample - loss: 0.2728 - accuracy: 0.5260
Epoch 2/2
1000/1000 [==============================] - 0s 36us/sample - loss: 0.2448 - accuracy: 0.5680
1000/1 [==========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 43us/sample - loss: 0.1147 - accuracy: 0.8900
Epoch 2/2
1000/1000 [==============================] - 0s 46us/sample - loss: 0.0815 - accuracy: 0.9040
1000/1 [==========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 51us/sample - loss: 0.0919 - accuracy: 0.8850
Epoch 2/2
1000/1000 [==============================] - 0s 35us/sample - loss: 0.0900 - accuracy: 0.8850
1000/1 [==========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 44us/sample - loss: 0.0918 - accuracy: 0.8830
Epoch 2/2
1000/1000 [==============================] - 0s 42us/sample - loss: 0.0906 - accuracy: 0.8830
1000/1 [==========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

...

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 51us/sample - loss: 0.0347 - accuracy: 0.9530
Epoch 2/2
1000/1000 [==============================] - 0s 46us/sample - loss: 0.0336 - accuracy: 0.9540
1000/1 [==========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 50us/sample - loss: 0.0106 - accuracy: 0.9870
Epoch 2/2
1000/1000 [==============================] - 0s 48us/sample - loss: 0.0105 - accuracy: 0.9850
1000/1 [==========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 52us/sample - loss: 0.0433 - accuracy: 0.9370
Epoch 2/2
1000/1000 [==============================] - 0s 51us/sample - loss: 0.0413 - accuracy: 0.9410
1000/1 [==========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 48us/sample - loss: 0.0410 - accuracy: 0.9520
Epoch 2/2
1000/1000 [==============================] - 0s 47us/sample - loss: 0.0402 - accuracy: 0.9530
1000/1 [==========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 0s 49us/sample - loss: 0.0305 - accuracy: 0.9590
Epoch 2/2
1000/1000 [==============================] - 0s 40us/sample - loss: 0.0311 - accuracy: 0.9610


In [110]:

print(accuracies)

100/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================